#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Define plot saving function

In [2]:
# Setting up default settings for plots, rc = Runtime configuration

plt.rc('figure',figsize = (10,6))          
plt.rc('font',size =14)                    # Defining font size for all texts (unless overriden)
plt.rc('xtick', labelsize = 14)            # you know what tick and tick_label is right?
plt.rc('ytick', labelsize = 14)
plt.rc('axes',labelsize= 14, titlesize= 16, titleweight = 'bold')   # Defining config of Xaxis, Yaxis and title

#### Get the dataset

In [8]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784',version=1)
X,y = np.array(mnist['data']), np.array(mnist['target'])
X.shape,y.shape

((70000, 784), (70000,))

#### Train-Test split

In [9]:
# Train-Test Split
X_train,X_test,y_train,y_test = X[:60000],X[60000:],y[:60000],y[60000:]

# _Multiclass Classification/Multinomial Classification_